In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
tf.set_random_seed(0)  
print ("TENSORFLOW VERSION IS %s" % (tf.__version__))

TENSORFLOW VERSION IS 1.12.0


In [2]:
## Define training sequence
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")
print ("FOLLOWING IS OUR TRAINING SEQUENCE:")
print (sentence)

FOLLOWING IS OUR TRAINING SEQUENCE:
if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.


In [3]:
## Define vocabulary and dictionary
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}
print ("VOCABULARY: ")
print (char_set)
print ("DICTIONARY: ")
print (char_dic)

VOCABULARY: 
[' ', 'b', ',', 'h', 'c', 'o', 'd', 'p', 'e', 't', 'i', 'g', "'", 's', 'f', 'n', 'w', '.', 'u', 'a', 'r', 'k', 'l', 'y', 'm']
DICTIONARY: 
{' ': 0, 'b': 1, ',': 2, 'h': 3, 'c': 4, 'o': 5, 'd': 6, 'p': 7, 'e': 8, 't': 9, 'i': 10, 'g': 11, "'": 12, 's': 13, 'f': 14, 'n': 15, 'w': 16, '.': 17, 'u': 18, 'a': 19, 'r': 20, 'k': 21, 'l': 22, 'y': 23, 'm': 24}


In [4]:
## Configure network (hyperparameter)
data_dim    = len(char_set)
num_classes = len(char_set)
hidden_size     = 64
sequence_length = 10  # Any arbitrary number

In [5]:
## Set training batches
dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index
    dataX.append(x)
    dataY.append(y)
    print ("[%3d/%3d] [%s]=>[%s]" % (i, len(sentence), x_str, y_str))
    print ("%s%s=>%s" % (' '*10, x, y))

[  0/180] [if you wan]=>[f you want]
          [10, 14, 0, 23, 5, 18, 0, 16, 19, 15]=>[14, 0, 23, 5, 18, 0, 16, 19, 15, 9]
[  1/180] [f you want]=>[ you want ]
          [14, 0, 23, 5, 18, 0, 16, 19, 15, 9]=>[0, 23, 5, 18, 0, 16, 19, 15, 9, 0]
[  2/180] [ you want ]=>[you want t]
          [0, 23, 5, 18, 0, 16, 19, 15, 9, 0]=>[23, 5, 18, 0, 16, 19, 15, 9, 0, 9]
[  3/180] [you want t]=>[ou want to]
          [23, 5, 18, 0, 16, 19, 15, 9, 0, 9]=>[5, 18, 0, 16, 19, 15, 9, 0, 9, 5]
[  4/180] [ou want to]=>[u want to ]
          [5, 18, 0, 16, 19, 15, 9, 0, 9, 5]=>[18, 0, 16, 19, 15, 9, 0, 9, 5, 0]
[  5/180] [u want to ]=>[ want to b]
          [18, 0, 16, 19, 15, 9, 0, 9, 5, 0]=>[0, 16, 19, 15, 9, 0, 9, 5, 0, 1]
[  6/180] [ want to b]=>[want to bu]
          [0, 16, 19, 15, 9, 0, 9, 5, 0, 1]=>[16, 19, 15, 9, 0, 9, 5, 0, 1, 18]
[  7/180] [want to bu]=>[ant to bui]
          [16, 19, 15, 9, 0, 9, 5, 0, 1, 18]=>[19, 15, 9, 0, 9, 5, 0, 1, 18, 10]
[  8/180] [ant to bui]=>[nt to buil]
          

In [6]:
ndata      = len(dataX)
batch_size = 20
print ("     'NDATA' IS %d" % (ndata))
print ("'BATCH_SIZE' IS %d" % (batch_size))

     'NDATA' IS 170
'BATCH_SIZE' IS 20


In [7]:
## Define placeholders (graph)
X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])
X_OH = tf.one_hot(X, num_classes)
print ("'sequence_length' IS [%d]" % (sequence_length))
print ("    'num_classes' IS [%d]" % (num_classes))
print("'X' LOOKS LIKE \n   [%s]" % (X))  
print("'X_OH' LOOKS LIKE \n   [%s]" % (X_OH))

'sequence_length' IS [10]
    'num_classes' IS [25]
'X' LOOKS LIKE 
   [Tensor("Placeholder:0", shape=(?, 10), dtype=int32)]
'X_OH' LOOKS LIKE 
   [Tensor("one_hot:0", shape=(?, 10, 25), dtype=float32)]


In [8]:
## Define model
with tf.variable_scope('CHAR-RNN', reuse=False):
    cell_1 = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    cell_2 = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    cell = rnn.MultiRNNCell([cell_1, cell_2], state_is_tuple=True)
    # DYNAMIC RNN WITH FULLY CONNECTED LAYER
    _outputs, _states = tf.nn.dynamic_rnn(cell, X_OH, dtype=tf.float32)
    _outputs  = tf.contrib.layers.fully_connected(_outputs, num_classes, activation_fn=None)
    # RESHAPE FOR SEQUNCE LOSS
    outputs = tf.reshape(_outputs, [batch_size, sequence_length, num_classes])
print ("OUTPUTS LOOKS LIKE [%s]" % (outputs))
print ("MODEL DEFINED.")

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
OUTPUTS LOOKS LIKE [Tensor("CHAR-RNN/Reshape:0", shape=(20, 10, 25), dtype=float32)]
MODEL DEFINED.


In [9]:
## Define TF Functions

# EQUAL WEIGHTS
weights = tf.ones([batch_size, sequence_length])
seq_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(seq_loss)
optm  = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)
print ("FUNCTIONS DEFINED.")

FUNCTIONS DEFINED.


In [10]:
## Optimize / Train

sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(500):
    randidx = np.random.randint(low=0, high=ndata, size=batch_size)
    batchX = [dataX[iii] for iii in randidx]
    batchY = [dataY[iii] for iii in randidx]
    feeds = {X: batchX, Y: batchY}
    _, loss_val, results = sess.run(
        [optm, loss, outputs], feed_dict=feeds)
    if (i%100) == 0:
        for j, result in enumerate(results):
            index = np.argmax(result, axis=1)
            print(i, j, ''.join([char_set[t] for t in index]), loss_val)

0 0 eitmmmmmmm 3.2195673
0 1 aawwwwwwww 3.2195673
0 2 pppmmmmllt 3.2195673
0 3 swl'lllmml 3.2195673
0 4 opyyyyywww 3.2195673
0 5 slylmmmmmm 3.2195673
0 6 pkccull''' 3.2195673
0 7 bggl''''ww 3.2195673
0 8 bgeeyywwww 3.2195673
0 9 riipppppww 3.2195673
0 10 i   wwbabb 3.2195673
0 11 pw''dddddd 3.2195673
0 12 oobmmmmmmm 3.2195673
0 13 efionnowww 3.2195673
0 14 oowwwwwwww 3.2195673
0 15 .ikwwaaadp 3.2195673
0 16 emmmmmmmmm 3.2195673
0 17 opppbmmb,, 3.2195673
0 18 opppbmmb,, 3.2195673
0 19 wwwwnnnnnn 3.2195673
100 0 t   t  t t 2.4361587
100 1 ett tt   t 2.4361587
100 2 t ttt tt   2.4361587
100 3  t t  t t  2.4361587
100 4 tt t t   t 2.4361587
100 5 t  t  tt t 2.4361587
100 6  t t  tt   2.4361587
100 7 tt  t t t  2.4361587
100 8 n t t  t t 2.4361587
100 9 n  tt   t  2.4361587
100 10     t t  t 2.4361587
100 11 t tt t  tt 2.4361587
100 12 t t t t tt 2.4361587
100 13 ett tt   t 2.4361587
100 14 n tt  t t  2.4361587
100 15  t tt  t t 2.4361587
100 16 n t t  t t 2.4361587
100 17 nt t t t t 2.4361

In [12]:
## Sampling Function
LEN = 1;
# XL = tf.placeholder(tf.int32, [None, LEN])
XL     = tf.placeholder(tf.int32, [None, 1])
XL_OH  = tf.one_hot(XL, num_classes)
with tf.variable_scope('CHAR-RNN', reuse=True):
    cell_L_1 = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    cell_L_2 = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    cell_L = rnn.MultiRNNCell([cell_L_1, cell_L_2], state_is_tuple=True)
    istate = cell_L.zero_state(batch_size=1, dtype=tf.float32)
    # DYNAMIC RNN WITH FULLY CONNECTED LAYER
    _outputs_L, states_L = tf.nn.dynamic_rnn(cell_L, XL_OH
                                , initial_state=istate, dtype=tf.float32)
    _outputs_L  = tf.contrib.layers.fully_connected(
        _outputs_L, num_classes, activation_fn=None)
    # RESHAPE FOR SEQUNCE LOSS
    outputs_L = tf.reshape(_outputs_L, [LEN, 1, num_classes])
print (XL)

Tensor("Placeholder_3:0", shape=(?, 1), dtype=int32)


In [13]:
## Sample

# BURNIN
prime = "if you "
istateval = sess.run(cell_L.zero_state(1, tf.float32))
for c in prime[:-1]:
    index = char_dic[c]
    inval = [[index]]
    outval, stateval = sess.run([outputs_L, states_L]
                        , feed_dict={XL:inval, istate:istateval})
    istateval = stateval

In [14]:
# SAMPLE
inval  = [[char_dic[prime[-1]]]]
outval, stateval = sess.run([outputs_L, states_L]
                    , feed_dict={XL:inval, istate:istateval})
index = np.argmax(outval)
char  = char_set[index]
chars = ''
for i in range(500):
    inval = [[index]]
    outval, stateval = sess.run([outputs_L, states_L]
                        , feed_dict={XL:inval, istate:istateval})
    istateval = stateval
    index = np.argmax(outval)
    char  = char_set[index]
    chars += char

print ("SAMPLED SETENCE: \n %s" % (prime+chars))
print ("\nORIGINAL SENTENCE: \n %s" % (sentence))

SAMPLED SETENCE: 
 if you don't drum up per together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together together toget

ORIGINAL SENTENCE: 
 if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
